In [32]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import nltk 
import string 
import sys
import numpy as np
import linalg
from sklearn import preprocessing
from sklearn.decomposition import TruncatedSVD

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

In [34]:
all_data = pd.read_csv("DataReady.csv")
all_data.shape

(5195, 11)

In [35]:
all_data.sample(10)

,Link,Data,Category,CategoryIndex,Keyword,VarKeyword,TotalFreqKeyword,TotalWord,TermRatio,interval,AVGInterval
5085,https://www.cia.gov/library/publications/the-w...,central america bahamas world factbook central...,Kids,1.0,"['cat', 'animal', 'tree', 'christmas', 'color'...",1123,5923,8547,0.692992,1553,0.0
1729,http://www.internet-colleges.net/,error page display contact service provider de...,General,-1.0,"['page', 'detail', 'service', 'provider', 'con...",6,6,7,0.857143,1579,263.0
3929,http://www.iidc.indiana.edu/cedir/kidsweb/fpwd...,cedir famous people disability skip navigation...,Kids,1.0,"['free', 'activity', 'word', 'music', 'learn',...",497,1080,1498,0.720961,1447,1.0
1052,http://www.bakingbites.com/,bake bite award win food bake blog feature rec...,General,-1.0,"['free', 'food', 'school', 'read', 'spell', 'p...",138,383,532,0.719925,1372,4.0
3780,http://www.kidsfreesouls.com/,kidsfreesouls newspaper kid resource parent te...,Kids,1.0,"['christmas', 'color', 'page', 'sheet', 'free'...",244,817,1511,0.540702,1608,2.0
2643,http://www.sportinglife.com/golf/news/,european tour pga tour bet preview tip news re...,General,-1.0,"['page', 'autumn', 'game', 'story', 'book', 'r...",133,279,532,0.524436,2133,8.0
3335,http://hea-www.harvard.edu/ECT/the_book/index....,eye sky foot grind collection child online ast...,Kids,1.0,"['page', 'activity', 'learn', 'teacher', 'site...",54,71,84,0.845238,946,13.0
162,http://www.stanford.edu/group/SHR/,stanford humanity review issn update april not...,General,-1.0,"['page', 'draw', 'science', 'site', 'productio...",201,304,482,0.630705,1162,4.0
1086,http://www.diyonline.com/,diyonline,General,-1.0,[],0,0,1,0.000000,0,NaN
714,http://greenethorne.tripod.com/,plaidscape find tripod site redirect click red...,General,-1.0,"['site', 'find', 'click']",3,3,7,0.428571,989,330.0


In [36]:
all_data.isnull().sum()

Link                 0
Data                 0
Category             0
CategoryIndex        0
Keyword              0
VarKeyword           0
TotalFreqKeyword     0
TotalWord            0
TermRatio            0
interval             0
AVGInterval         48
dtype: int64

In [37]:
clean_data = all_data
clean_data["AVGInterval"] = clean_data['AVGInterval'].replace(np.nan, 0)
clean_data.isnull().sum()

Link                0
Data                0
Category            0
CategoryIndex       0
Keyword             0
VarKeyword          0
TotalFreqKeyword    0
TotalWord           0
TermRatio           0
interval            0
AVGInterval         0
dtype: int64

In [38]:
clean_data.head(5)

,Link,Data,Category,CategoryIndex,Keyword,VarKeyword,TotalFreqKeyword,TotalWord,TermRatio,interval,AVGInterval
0,http://benjah.free.fr/,benjah free rvb notv contrat creative common,General,-1.0,"['free', 'creative', 'common']",3,3,7,0.428571,1598,533.0
1,http://www.paaw.com/,perform art artist worldwide perform art artis...,General,-1.0,"['site', 'production', 'web', 'project', 'read...",80,144,188,0.765957,1568,11.0
2,http://vlib.org/Humanities,humanity humanistic study virtual library huma...,General,-1.0,"['page', 'science', 'online', 'high', 'type', ...",60,145,219,0.662100,2063,14.0
3,http://www.berzowska.com/xy/index.html,computational expressionism,General,-1.0,['expressionism'],1,1,2,0.500000,0,0.0
4,http://www.chronotext.org/,chronotext org chronotext grow collection soft...,General,-1.0,"['tree', 'word', 'poem', 'music', 'site', 'gam...",256,476,677,0.703102,1608,3.0


In [39]:
clean_data['Category'].value_counts()

General    2684
Kids       2511
Name: Category, dtype: int64

In [40]:
# MEMBUAT DATA FRAME DENGAN ISI KOLOM YANG DIGUNAKAN SAJAA
x_data = clean_data[["Data", "TotalFreqKeyword", "TotalWord", "TermRatio", "AVGInterval"]]
y_data = clean_data["CategoryIndex"]
x_data.shape, y_data.shape

((5195, 5), (5195,))

In [41]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=3)

print (x_train.shape , y_train.shape)
print (x_test.shape, y_test.shape)

(3636, 5) (3636,)
(1559, 5) (1559,)


In [42]:
x_train.sample(10)

,Data,TotalFreqKeyword,TotalWord,TermRatio,AVGInterval
1279,newamericangazette justhost web host host home...,24,31,0.774194,53.0
3436,girl summer art camp dance camp music camp the...,530,737,0.719132,2.0
2515,foxsports tennishttp foxsports tennis latest t...,175,369,0.474255,14.0
4067,kid web india portal kid young teen kid web si...,352,432,0.814815,5.0
4834,bahamas fact history news toggle navigation se...,366,482,0.759336,4.0
3688,ornithopter zone build ornithopter ornithopter...,490,634,0.772871,3.0
3089,draw lesson draw cartoon animal monster alien ...,257,315,0.815873,5.0
3693,kid genealogyspot back home cemetery census fe...,170,222,0.765766,7.0
525,copyright vrije universiteit netherlands reser...,46,88,0.522727,21.0
4928,bioterrorism real imagine bioterrorism real im...,243,396,0.613636,9.0


In [43]:
#LATENT SEMANTIC ANALYSIS

x_train_lsa = x_train["Data"]
y_train_lsa = y_train
x_test_lsa = x_test["Data"]
y_test_lsa = y_test

print(x_train_lsa.shape, y_train_lsa.shape)
print(x_test_lsa.shape, y_test_lsa.shape)

(3636,) (3636,)
(1559,) (1559,)


In [44]:
#vectorizer = CountVectorizer(max_df=0.9, min_df=5)
vectorizer = TfidfVectorizer(max_df=0.9, min_df=5)
x_train_lsa_vec = vectorizer.fit_transform(x_train_lsa).todense()
x_test_lsa_vec = vectorizer.fit_transform(x_test_lsa).todense()
#X_tf = tfidf.fit_transform(x_train).todense()
#print (X_tf)

In [45]:
x_train_lsa_vec.shape, x_test_lsa_vec.shape

((3636, 16906), (1559, 10303))

In [46]:
svd = TruncatedSVD(n_components = 100)
lsa = make_pipeline(svd, Normalizer(copy=False))

x_transform_lsa_train = lsa.fit_transform(x_train_lsa_vec)
x_transform_lsa_test = lsa.fit_transform(x_test_lsa_vec)

U_matrix_train = x_transform_lsa_train / svd.singular_values_
S_matrix_train = np.diag(svd.singular_values_)
VT_matrix_train = svd.components_

U_matrix_test = x_transform_lsa_test / svd.singular_values_
S_matrix_test = np.diag(svd.singular_values_)
VT_matrix_test = svd.components_


In [47]:
print("Train Data 2: ", x_transform_lsa_train.shape, U_matrix_train.shape, S_matrix_train.shape, VT_matrix_train.shape)
print("Test Data 2: ", x_transform_lsa_test.shape, U_matrix_test.shape, S_matrix_test.shape, VT_matrix_test.shape)

Train Data 2:  (3636, 100) (3636, 100) (100, 100) (100, 10303)
Test Data 2:  (1559, 100) (1559, 100) (100, 100) (100, 10303)


In [48]:
sem_matrix_train_2 = np.dot(U_matrix_train, S_matrix_train)
sem_matrix_test_2 = np.dot(U_matrix_test, S_matrix_test)

sem_matrix_train_2.shape , sem_matrix_test_2.shape

((3636, 100), (1559, 100))

In [49]:
from sklearn import svm
svm_model_lsa_linear = svm.SVC(kernel='linear', gamma=1, probability=True)
svm_model_lsa_linear.fit(sem_matrix_train_2, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [50]:
svm_model_lsa_poly = svm.SVC(kernel='poly', gamma=1, probability=True)
svm_model_lsa_poly.fit(sem_matrix_train_2, y_train) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='poly',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [51]:
svm_model_lsa_rbf = svm.SVC(kernel='rbf', gamma=1, probability=True)
svm_model_lsa_rbf.fit(sem_matrix_train_2, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [52]:
svm_model_lsa_sigmoid = svm.SVC(kernel='sigmoid', gamma=1, probability=True)
svm_model_lsa_sigmoid.fit(sem_matrix_train_2, y_train) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='sigmoid',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [53]:
# WEB PAGE FEATURE SELECTION

x_train_wpfs = x_train[["TotalFreqKeyword", "TotalWord", "TermRatio", "AVGInterval"]]
y_train_wpfs = y_train
x_test_wpfs = x_test[["TotalFreqKeyword", "TotalWord", "TermRatio", "AVGInterval"]]
y_test_wpfs = y_test

x_train_wpfs = preprocessing.scale(x_train_wpfs)
x_test_wpfs = preprocessing.scale(x_test_wpfs)

print(x_train_wpfs.shape, y_train_wpfs.shape)
print(x_test_wpfs.shape, y_test_wpfs.shape)

(3636, 4) (3636,)
(1559, 4) (1559,)


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  if __name__ == '__main__':


In [54]:
svm_model_wpfs_linear = svm.SVC(kernel='linear', gamma=1, probability=True)
svm_model_wpfs_linear.fit(x_train_wpfs, y_train) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [55]:
svm_model_wpfs_poly = svm.SVC(kernel='poly', gamma=1, probability=True)
svm_model_wpfs_poly.fit(x_train_wpfs, y_train) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='poly',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [56]:
svm_model_wpfs_rbf = svm.SVC(kernel='rbf', gamma=1, probability=True)
svm_model_wpfs_rbf.fit(x_train_wpfs, y_train) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [57]:
svm_model_wpfs_sigmoid = svm.SVC(kernel='sigmoid', gamma=1, probability=True)
svm_model_wpfs_sigmoid.fit(x_train_wpfs, y_train) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='sigmoid',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [58]:
lsa_value_linear = svm_model_lsa_linear.decision_function(sem_matrix_test_2)
lsa_value_poly = svm_model_lsa_poly.decision_function(sem_matrix_test_2)
lsa_value_rbf = svm_model_lsa_rbf.decision_function(sem_matrix_test_2)
lsa_value_sigmoid = svm_model_lsa_sigmoid.decision_function(sem_matrix_test_2)

lsa_predict_linear = svm_model_lsa_linear.predict(sem_matrix_test_2)
lsa_predict_poly = svm_model_lsa_poly.predict(sem_matrix_test_2)
lsa_predict_rbf = svm_model_lsa_rbf.predict(sem_matrix_test_2)
lsa_predict_sigmoid = svm_model_lsa_sigmoid.predict(sem_matrix_test_2)

wpfs_value_linear = svm_model_wpfs_linear.decision_function(x_test_wpfs)
wpfs_value_poly = svm_model_wpfs_poly.decision_function(x_test_wpfs)
wpfs_value_rbf = svm_model_wpfs_rbf.decision_function(x_test_wpfs)
wpfs_value_sigmoid = svm_model_wpfs_sigmoid.decision_function(x_test_wpfs)

wpfs_predict_linear = svm_model_wpfs_linear.predict(x_test_wpfs)
wpfs_predict_poly = svm_model_wpfs_poly.predict(x_test_wpfs)
wpfs_predict_rbf = svm_model_wpfs_rbf.predict(x_test_wpfs)
wpfs_predict_sigmoid = svm_model_wpfs_sigmoid.predict(x_test_wpfs)

In [59]:
list_decision_linear = list(zip(lsa_value_linear, lsa_predict_linear, wpfs_value_linear, wpfs_predict_linear))    
list_decision_poly = list(zip(lsa_value_poly, lsa_predict_poly, wpfs_value_poly, wpfs_predict_poly))    
list_decision_rbf = list(zip(lsa_value_rbf, lsa_predict_rbf, wpfs_value_rbf, wpfs_predict_rbf))    
list_decision_sigmoid = list(zip(lsa_value_sigmoid, lsa_predict_sigmoid, wpfs_value_sigmoid, wpfs_predict_sigmoid))    


DF_linear = pd.DataFrame(list_decision_linear, columns = ['LSA_val_lin','LSA_pred_lin', 'WPFS_val_lin', 'WPFS_pred_lin'])
DF_poly = pd.DataFrame(list_decision_poly, columns = ['LSA_val_pol','LSA_pred_pol', 'WPFS_val_pol', 'WPFS_pred_pol'])
DF_rbf = pd.DataFrame(list_decision_rbf, columns = ['LSA_val_rbf','LSA_pred_rbf', 'WPFS_val_rbf', 'WPFS_pred_rbf'])
DF_sigmoid = pd.DataFrame(list_decision_sigmoid, columns = ['LSA_val_sig','LSA_pred_sig', 'WPFS_val_sig', 'WPFS_pred_sig'])

In [60]:
count_linear = DF_linear["LSA_val_lin"]
count_poly = DF_poly["LSA_val_pol"]
count_rbf = DF_rbf["LSA_val_rbf"]
count_sigmoid = DF_sigmoid["LSA_val_sig"]


for n in count_linear:
    if DF_linear['LSA_pred_lin'].equals(DF_linear['WPFS_pred_lin']):
        DF_linear["Final_Category_lin"] = DF_linear["LSA_pred_lin"]
    else:
        lsaval = abs(DF_linear["LSA_val_lin"])
        wpfsval = abs(DF_linear["WPFS_val_lin"])
        DF_linear['Final_Category_lin'] = np.where((lsaval > wpfsval), DF_linear["LSA_pred_lin"],  DF_linear["WPFS_pred_lin"])

for n in count_poly:
    if DF_poly['LSA_pred_pol'].equals(DF_poly['WPFS_pred_pol']):
        DF_poly["Final_Category_pol"] = DF_poly["LSA_pred_pol"]
    else:
        lsaval = abs(DF_poly["LSA_val_pol"])
        wpfsval = abs(DF_poly["WPFS_val_pol"])
        DF_poly['Final_Category_pol'] = np.where((lsaval > wpfsval), DF_poly["LSA_pred_pol"],  DF_poly["WPFS_pred_pol"])

for n in count_rbf:
    if DF_rbf['LSA_pred_rbf'].equals(DF_rbf['WPFS_pred_rbf']):
        DF_rbf["Final_Category_rbf"] = DF_rbf["LSA_predict"]
    else:
        lsaval = abs(DF_rbf["LSA_val_rbf"])
        wpfsval = abs(DF_rbf["WPFS_val_rbf"])
        DF_rbf['Final_Category_rbf'] = np.where((lsaval > wpfsval), DF_rbf["LSA_pred_rbf"],  DF_rbf["WPFS_pred_rbf"])

for n in count_sigmoid:
    if DF_sigmoid['LSA_pred_sig'].equals(DF_sigmoid['WPFS_pred_sig']):
        DF_sigmoid["Final_Category_sig"] = DF_sigmoid["LSA_pred_sig"]
    else:
        lsaval = abs(DF_sigmoid["LSA_val_sig"])
        wpfsval = abs(DF_sigmoid["WPFS_val_sig"])
        DF_sigmoid['Final_Category_sig'] = np.where((lsaval > wpfsval), DF_sigmoid["LSA_pred_sig"],  DF_sigmoid["WPFS_pred_sig"])
        


In [61]:
y_pred_lin = DF_linear["Final_Category_lin"]
y_pred_pol = DF_poly['Final_Category_pol']
y_pred_rbf = DF_rbf['Final_Category_rbf']
y_pred_sig = DF_sigmoid['Final_Category_sig']

In [62]:
from sklearn import metrics 
from sklearn.metrics import classification_report, confusion_matrix
import itertools

acc_lin = metrics.accuracy_score(y_test, y_pred_lin)
acc_pol = metrics.accuracy_score(y_test, y_pred_pol)
acc_rbf = metrics.accuracy_score(y_test, y_pred_rbf)
acc_sig = metrics.accuracy_score(y_test, y_pred_sig)

print("=========================LINEAR KERNEL=========================")
print (classification_report(y_test, y_pred_lin))
print("akurasi : ", acc_lin)

print("=========================POLINOMIAL KERNEL=========================")
print (classification_report(y_test, y_pred_pol))
print("akurasi : ", acc_pol)

print("=========================RBF KERNEL=========================")
print (classification_report(y_test, y_pred_rbf))
print("akurasi : ", acc_rbf)

print("=========================SIGMOID KERNEL=========================")
print (classification_report(y_test, y_pred_sig))
print("akurasi : ", acc_sig)

=========================LINEAR KERNEL=========================
              precision    recall  f1-score   support

        -1.0       0.68      0.67      0.68       820
         1.0       0.64      0.65      0.65       739

   micro avg       0.66      0.66      0.66      1559
   macro avg       0.66      0.66      0.66      1559
weighted avg       0.66      0.66      0.66      1559

akurasi :  0.6626042334830019
=========================POLINOMIAL KERNEL=========================
              precision    recall  f1-score   support

        -1.0       0.57      0.97      0.72       820
         1.0       0.85      0.19      0.31       739

   micro avg       0.60      0.60      0.60      1559
   macro avg       0.71      0.58      0.52      1559
weighted avg       0.70      0.60      0.53      1559

akurasi :  0.600384862091084
=========================RBF KERNEL=========================
              precision    recall  f1-score   support

        -1.0       0.71      0.77      